In [11]:
import openpyxl

def insert_differences(data):
    result = []
    # Start from the second column (index 1)
    for i in range(1, len(data) - 1, 2):
        first_value = data[i] if data[i] is not None else "N/A"
        second_value = data[i+1] if data[i+1] is not None else "N/A"

        try:
            first_num = float(first_value)
            second_num = float(second_value)
            difference = second_num - first_num
        except ValueError:
            difference = "N/A"

        result.append(first_value)
        result.append(second_value)
        result.append("N/A")  # Insert placeholder to maintain two-column spacing
        result.append("N/A")  # Insert placeholder to maintain two-column spacing
        result.append(difference)

    if len(data) % 2 == 0:
        last_value = data[-1] if data[-1] is not None else "N/A"
        result.append(last_value)
    return result

def process_and_sort_rows(sheet, header_order):
    processed_rows = []
    not_found_rows = []

    headers = [cell.value for cell in sheet[1]]  # Assuming the first row is the header
    header_index_map = {header: i for i, header in enumerate(headers)}

    # Create a new header row with "Difference" columns spaced with two columns in between
    new_headers = ["Header"] + headers[1:]
    for i in range(1, len(new_headers) - 1, 2):
        new_headers.insert(i + 4, "Difference")  # Insert "Difference" with two-column spacing

    # Clear the old header row
    for cell in sheet[1]:
        cell.value = None
    
    # Write the new header row
    for j, value in enumerate(new_headers):
        sheet.cell(row=1, column=j+1).value = value

    for row in sheet.iter_rows(min_row=2, max_col=sheet.max_column, max_row=sheet.max_row, values_only=True):
        header_value = row[0]  # Assuming the first column is the header for each row
        processed_row = [header_value] + insert_differences(list(row))

        if header_value in header_order:
            index = header_order.index(header_value)
            processed_rows.append((index, processed_row))
        else:
            not_found_rows.append(["couldn't find"] + processed_row[1:])  # Exclude the original header column from not found rows

    processed_rows.sort(key=lambda x: x[0])
    sorted_data = [row for _, row in processed_rows]
    sorted_data.extend(not_found_rows)

    for row in sheet.iter_rows(min_row=2, max_col=sheet.max_column, max_row=sheet.max_row):
        for cell in row:
            cell.value = None

    for i, row_data in enumerate(sorted_data, start=2):
        for j, value in enumerate(row_data):
            sheet.cell(row=i, column=j+1).value = value



In [13]:
import pandas as pd

# Read the Excel file
excel_file = 'input.xlsx'
df = pd.read_excel(excel_file)

# Calculate the percentage differences and add new columns
df['Teacher/Adult Cares Diff'] = df['Teacher/Adult Cares'].diff()
df['Happy to be at School Diff'] = df['Happy to be at School'].diff()

# Save the updated DataFrame to a new Excel file
output_file = 'output.xlsx'
df.to_excel(output_file, index=False)

print(f"Data saved to {output_file}")

KeyError: 'Teacher/Adult Cares'